A model has been created and saved to a pickle file. Load the model, get new data, and predict on the new data.

In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import sys, os
import joblib
from sklearn.preprocessing import StandardScaler

# Get the current working directory
current_dir = os.getcwd()

# Get the path to the parent directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add the parent directory to the Python path
sys.path.insert(0, parent_dir)
import indicators as ind 

In [29]:
symbol_list = ['GOOGL', 'AMC', 'WMT', 'V', 'TNDM', 'DNB', 'RARE', 'TSLA', 'AAPL', 'MSFT', 'NFLX', 'AMZN', 'GOOGL', 'META', 'SNAP', 'PINS', 'SPOT', 'T', 'VZ', 'TMUS', 'S', 'TM', 'F', 'GM', 'HMC']
# symbol_list = ['DOGE-USD']

for symbol in symbol_list:

    # get data from yfinance
    df = yf.download(symbol, start='2014-09-19', end='2024-09-19', interval='1d')

    if df.empty:
        print(f"No data found for {symbol}")
        continue
    else:
        print(df.shape)

    # rename columns
    df = df.rename(columns={'Close': 'close', 'High': 'high', 'Low': 'low', 'Open': 'open', 'Volume': 'volume'})

    # log normalize price data
    df['log_close'] = df['close'].apply(lambda x: np.log(x + 1))
    df['log_high'] = df['high'].apply(lambda x: np.log(x + 1))
    df['log_low'] = df['low'].apply(lambda x: np.log(x + 1))
    df['log_open'] = df['open'].apply(lambda x: np.log(x + 1))

    # log normalize volume
    df['log_volume'] = df['volume'].apply(lambda x: np.log(x + 1))

    # calculate volume average
    df['volume_ma'] = df['volume'].rolling(window=20).mean()

    # log normalize volume_ma
    df['log_volume_ma'] = df['volume_ma'].apply(lambda x: np.log(x + 1))

    df = ind.calculate_moving_averages(df).copy()

    # log normalize ma fields
    df['log_ma_10'] = df['ma_10'].apply(lambda x: np.log(x + 1))
    df['log_ma_50'] = df['ma_50'].apply(lambda x: np.log(x + 1))
    df['log_ma_100'] = df['ma_100'].apply(lambda x: np.log(x + 1))
    df['log_ma_200'] = df['ma_200'].apply(lambda x: np.log(x + 1))

    df = ind.calculate_adx(df).copy()

    # log normalize adx
    df['log_adx'] = df['adx'].apply(lambda x: np.log(x + 1))

    # drop first 200 rows
    df = df.iloc[200:]

    # Extract necessary columns for predictions
    pred_df = df[['log_close', 'log_high', 'log_low', 'log_open', 'log_adx', 
                  'log_volume', 'log_volume_ma', 'log_ma_10', 'log_ma_50', 
                  'log_ma_100', 'log_ma_200']]
    
    # Load the saved scaler
    scaler = joblib.load('scaler.pkl')

    # Scale the entire dataset
    pred_data_scaled = scaler.transform(pred_df)

    # Load the model from the file
    loaded_rf_model = joblib.load('random_forest_model.pkl')

    # Make predictions for all rows
    predictions = loaded_rf_model.predict(pred_data_scaled)

    # Add the predictions as a new column to the original DataFrame
    df['Prediction'] = predictions

    # Optionally, filter for records where the prediction is True
    profitable_trades = df[df['Prediction'] == True]

    # Print the dates and predictions
    print(profitable_trades[['Prediction']])


[*********************100%%**********************]  1 of 1 completed


(2516, 6)
Empty DataFrame
Columns: [Prediction]
Index: []


[*********************100%%**********************]  1 of 1 completed


(2516, 6)
            Prediction
Date                  
2015-08-05        True
2015-08-06        True
2015-08-07        True
2015-08-10        True
2015-08-11        True
...                ...
2018-11-02        True
2018-11-05        True
2019-01-11        True
2019-04-11        True
2020-04-15        True

[64 rows x 1 columns]


[*********************100%%**********************]  1 of 1 completed


...

(2516, 6)
Empty DataFrame
Columns: [Prediction]
Index: []


[*********************100%%**********************]  1 of 1 completed


(2516, 6)
Empty DataFrame
Columns: [Prediction]
Index: []


[*********************100%%**********************]  1 of 1 completed

(2516, 6)
            Prediction
Date                  
2016-03-21        True
2016-03-23        True
2016-03-28        True
2016-03-29        True
2016-03-31        True
...                ...
2023-07-20        True
2023-07-24        True
2023-07-25        True
2023-07-27        True
2023-08-25        True

[343 rows x 1 columns]
